<a href="https://colab.research.google.com/github/akannihafeez123-hue/Kleez/blob/main/Kleezbot_Ultra_Retrain_Integrated_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os

# Simulate .env file loading by setting environment variables directly
# Replace the placeholder values with your actual secrets

os.environ["TELEGRAM_BOT_TOKEN"] = "YOUR_TELEGRAM_BOT_TOKEN"
os.environ["TELEGRAM_ADMIN_ID"] = "YOUR_ADMIN_ID"
os.environ["TELEGRAM_AUTHORIZED_USERS"] = "YOUR_AUTHORIZED_USERS" # Comma-separated IDs if multiple
os.environ["BITGET_API_KEY"] = "YOUR_BITGET_API_KEY"
os.environ["BITGET_API_SECRET"] = "YOUR_BITGET_API_SECRET"
os.environ["BITGET_PWD"] = "YOUR_BITGET_PWD"
os.environ["OWNER_SECRET"] = "YOUR_OWNER_SECRET"

print("Environment variables set for this session.")

Environment variables set for this session.


In [2]:
# =========================================================
# 🔥 INSTITUTIONAL ULTRA ENGINE — Bitget (Full) + Telegram Polling
# =========================================================
!pip install --quiet ccxt python-telegram-bot==20.6 nest_asyncio

import os, time, json, asyncio, math, traceback
from datetime import datetime, timedelta, timezone
from collections import defaultdict
import numpy as np
import pandas as pd
import ccxt
import nest_asyncio
nest_asyncio.apply()

from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes

# -------------------------
# CONFIG — set these before running (or set env vars)
# -------------------------
BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN", "")              # Telegram Bot token
ADMIN_ID = int(os.getenv("TELEGRAM_ADMIN_ID", "0"))         # Your Telegram numeric ID
AUTHORIZED_USERS = [int(id) for id in os.getenv("TELEGRAM_AUTHORIZED_USERS", "").split(",") if id]                      # Add others if needed

BITGET_API_KEY    = os.getenv("BITGET_API_KEY", "")
BITGET_API_SECRET = os.getenv("BITGET_API_SECRET", "")
BITGET_PWD        = os.getenv("BITGET_PWD", "")
OWNER_SECRET      = os.getenv("OWNER_SECRET", "")

# PARAMETERS (tweakable)
ALIGNMENT_THRESHOLD = 0.80
MIN_TF_COUNT = 4
MIN_STRAT_COUNT = 4
ANALYSIS_TFS = ["1m","5m","15m","30m","1h","4h","1d"]   # practical TFS for ccxt fetches
DAILY_RETRAIN_HOUR_UTC = 0

# DATA PATHS
os.makedirs("/mnt/data", exist_ok=True)
CANDLE_STATS_PATH = "/mnt/data/candle_stats.json"
TRADE_HISTORY_PATH = "/mnt/data/trade_history.json"
WEIGHTS_PATH = "/mnt/data/strategy_timeframe_weights.json"
RETRAIN_LOG_PATH = "/mnt/data/retrain_log.json"

# Ensure files exist
def _ensure(path, default):
    if not os.path.exists(path):
        with open(path,"w") as f:
            json.dump(default, f, indent=2)
_ensure(CANDLE_STATS_PATH, {})
_ensure(TRADE_HISTORY_PATH, [])
_ensure(WEIGHTS_PATH, {})
_ensure(RETRAIN_LOG_PATH, {})

# -------------------------
# Bitget / ccxt client (demo fallback)
# -------------------------
class BitgetClient:
    def __init__(self, api_key, api_secret, api_pwd):
        self.demo = (not api_key) or (not api_secret)
        self.exchange = None
        if not self.demo:
            try:
                self.exchange = ccxt.bitget({
                    "apiKey": api_key,
                    "secret": api_secret,
                    "password": api_pwd,
                    "enableRateLimit": True,
                    "options": {"defaultType":"swap"}
                })
                # load markets for symbol list
                self.exchange.load_markets()
                print("✅ Bitget live client initialized.")
            except Exception as e:
                print("⚠️ Bitget init failed, switching to DEMO. Error:", e)
                self.demo = True
        else:
            print("⚠️ Bitget running in DEMO mode (no keys).")

    def list_symbols(self):
        if self.exchange:
            try:
                return list(self.exchange.markets.keys())
            except:
                return []
        # Demo: some default markets
        return ["BTC/USDT:USDT","ETH/USDT:USDT","XAU/USDT:USDT","EUR/USDT:USDT"]

    def get_klines(self, symbol, timeframe, limit=200):
        # try live fetch
        if self.exchange and not self.demo:
            try:
                ohlcv = self.exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
                df = pd.DataFrame(ohlcv, columns=["timestamp","open","high","low","close","volume"])
                df["open_time"] = pd.to_datetime(df["timestamp"], unit="ms")
                df.drop(columns=["timestamp"], inplace=True)
                return df
            except Exception as e:
                # show one-time warning then fallback
                print(f"Warning fetching {symbol} {timeframe}:", e)
        # DEMO: synthetic data
        end = pd.Timestamp.now(tz=timezone.utc)
        if timeframe.endswith('m'):
            minutes = int(timeframe[:-1])
            delta = pd.Timedelta(minutes, 'minutes')
            freq = f"{minutes}min"
        elif timeframe.endswith('h'):
            hours = int(timeframe[:-1])
            delta = pd.Timedelta(hours, 'hours')
            freq = f"{hours}H"
        elif timeframe.endswith('d'):
            days = int(timeframe[:-1])
            delta = pd.Timedelta(days, 'days')
            freq = f"{days}D"
        else:
            delta = pd.Timedelta(1, 'hours'); freq = "1H"
        dates = pd.date_range(end - delta*limit, periods=limit, freq=freq)
        df = pd.DataFrame({
            "open_time": dates,
            "open": np.random.random(len(dates))*100 + 10000,
            "high": np.random.random(len(dates))*100 + 10100,
            "low": np.random.random(len(dates))*100 + 9900,
            "close": np.random.random(len(dates))*100 + 10050,
            "volume": np.random.random(len(dates))*1000
        })
        return df

    def create_market_order(self, symbol, side, qty, params=None):
        if self.demo or not self.exchange:
            return {"status":"demo","symbol":symbol,"side":side,"qty":qty,"params":params}
        try:
            # NOTE: ccxt param names differ by exchange & may require test net adjustments
            return self.exchange.create_market_order(symbol, side, qty, params=params)
        except Exception as e:
            return {"status":"error","error": str(e)}

bitget = BitgetClient(BITGET_API_KEY, BITGET_API_SECRET, BITGET_PWD)

# -------------------------
# Helpers for file persistence
# -------------------------
def safe_load(path, default):
    try:
        with open(path,"r") as f:
            return json.load(f)
    except:
        return default

def safe_save(path, obj):
    with open(path,"w") as f:
        json.dump(obj, f, indent=2)

# -------------------------
# Indicators
# -------------------------
def add_institutional_indicators(df):
    df = df.copy()
    # ensure numeric
    for c in ["open","high","low","close","volume"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')
    # EMAs
    df["ema_9"] = df["close"].ewm(span=9).mean()
    df["ema_21"] = df["close"].ewm(span=21).mean()
    df["ema_50"] = df["close"].ewm(span=50).mean()
    # RSI
    delta = df["close"].diff()
    gain = delta.clip(lower=0); loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(14).mean(); avg_loss = loss.rolling(14).mean()
    rs = avg_gain / (avg_loss + 1e-9)
    df["rsi"] = 100 - (100 / (1 + rs))
    # MACD
    ema12 = df["close"].ewm(span=12).mean(); ema26 = df["close"].ewm(span=26).mean()
    df["macd"] = ema12 - ema26; df["macd_signal"] = df["macd"].ewm(span=9).mean()
    # Bollinger
    sma20 = df["close"].rolling(20).mean(); std20 = df["close"].rolling(20).std()
    df["bb_upper"] = sma20 + 2*std20; df["bb_lower"] = sma20 - 2*std20
    # ATR
    tr = pd.concat([df["high"]-df["low"], (df["high"]-df["close"].shift()).abs(), (df["low"]-df["close"].shift()).abs()], axis=1).max(axis=1)
    df["atr"] = tr.rolling(14).mean()
    # Supertrend approximation
    hl2 = (df["high"] + df["low"]) / 2; factor = 3.0
    df["supertrend_upper"] = hl2 + factor*df["atr"]; df["supertrend_lower"] = hl2 - factor*df["atr"]
    # Ichimoku simplified
    df['ichimoku_tenkan_sen'] = (df['high'].rolling(9).max() + df['low'].rolling(9).min())/2
    df['ichimoku_kijun_sen'] = (df['high'].rolling(26).max() + df['low'].rolling(26).min())/2
    df['ichimoku_senkou_span_a'] = ((df['ichimoku_tenkan_sen'] + df['ichimoku_kijun_sen'])/2).shift(26)
    df['ichimoku_senkou_span_b'] = ((df['high'].rolling(52).max() + df['low'].rolling(52).min())/2).shift(26)
    # Stochastic
    df['stoch_k'] = 100 * ((df['close'] - df['low'].rolling(14).min()) / (df['high'].rolling(14).max() - df['low'].rolling(14).min() + 1e-9))
    df['stoch_d'] = df['stoch_k'].rolling(3).mean()
    # VWAP/OBV/ADL
    if 'volume' in df.columns:
        df['vwap'] = (df['close'] * df['volume']).cumsum() / (df['volume'].cumsum() + 1e-9)
        obv_dir = np.sign(df['close'].diff()).fillna(0)
        df['obv'] = (df['volume'] * obv_dir).cumsum()
        clv = ((df['close'] - df['low']) - (df['high'] - df['close'])) / (df['high'] - df['low'] + 1e-9)
        df['adl'] = (clv * df['volume']).cumsum()
    else:
        df['vwap'] = df['obv'] = df['adl'] = np.nan
    df.fillna(method='bfill', inplace=True); df.fillna(method='ffill', inplace=True)
    return df

# -------------------------
# Candle Learning Module (detect + scoring + record)
# -------------------------
DEFAULT_PATTERNS = {
    "bullish_engulfing": {"success_rate":0.5,"count":0,"wins":0,"losses":0},
    "bearish_engulfing": {"success_rate":0.5,"count":0,"wins":0,"losses":0},
    "hammer": {"success_rate":0.5,"count":0,"wins":0,"losses":0},
    "hanging_man": {"success_rate":0.5,"count":0,"wins":0,"losses":0},
    "pinbar_upper": {"success_rate":0.5,"count":0,"wins":0,"losses":0},
    "pinbar_lower": {"success_rate":0.5,"count":0,"wins":0,"losses":0},
    "doji": {"success_rate":0.5,"count":0,"wins":0,"losses":0},
    "marubozu": {"success_rate":0.5,"count":0,"wins":0,"losses":0},
    "tweezers_top": {"success_rate":0.5,"count":0,"wins":0,"losses":0},
    "tweezers_bottom": {"success_rate":0.5,"count":0,"wins":0,"losses":0}
}
# initialize if empty
if not safe_load(CANDLE_STATS_PATH, {}):
    safe_save(CANDLE_STATS_PATH, DEFAULT_PATTERNS)

def detect_patterns_from_df(df):
    last = df.iloc[-3:].copy().reset_index(drop=True)
    patterns = defaultdict(int)
    last['body'] = (last['close'] - last['open']).abs()
    last['upper_wick'] = last['high'] - last[['close','open']].max(axis=1)
    last['lower_wick'] = last[['close','open']].min(axis=1) - last['low']
    last['range'] = last['high'] - last['low'] + 1e-9
    last['body_ratio'] = last['body'] / last['range']
    # doji
    if last.loc[2,'body_ratio'] <= 0.1: patterns['doji'] += 1
    # hammer/hanging
    if last.loc[2,'body_ratio'] < 0.4 and last.loc[2,'lower_wick'] > 2 * last.loc[2,'body']:
        if last.loc[2,'close'] > last.loc[2,'open']: patterns['hammer'] += 1
        else: patterns['hanging_man'] += 1
    # engulfing
    if (last.loc[1,'close'] < last.loc[1,'open']) and (last.loc[2,'close'] > last.loc[2,'open']) and (last.loc[2,'open'] < last.loc[1,'close']) and (last.loc[2,'close'] > last.loc[1,'open']):
        patterns['bullish_engulfing'] += 1
    if (last.loc[1,'close'] > last.loc[1,'open']) and (last.loc[2,'close'] < last.loc[2,'open']) and (last.loc[2,'open'] > last.loc[1,'close']) and (last.loc[2,'close'] < last.loc[1,'open']):
        patterns['bearish_engulfing'] += 1
    # pinbar
    if last.loc[2,'upper_wick'] > 3 * last.loc[2,'body']: patterns['pinbar_upper'] += 1
    if last.loc[2,'lower_wick'] > 3 * last.loc[2,'body']: patterns['pinbar_lower'] += 1
    # marubozu
    if last.loc[2,'upper_wick'] < 0.05*last.loc[2,'range'] and last.loc[2,'lower_wick'] < 0.05*last.loc[2,'range'] and last.loc[2,'body_ratio']>0.8:
        patterns['marubozu'] += 1
    # tweezers
    if abs(last.loc[1,'low'] - last.loc[2,'low'])/max(1e-9,last.loc[1,'low']) < 0.002: patterns['tweezers_bottom'] += 1
    if abs(last.loc[1,'high'] - last.loc[2,'high'])/max(1e-9,last.loc[1,'high']) < 0.002: patterns['tweezers_top'] += 1
    return dict(patterns)

def candle_confidence_from_patterns(patterns):
    stats = safe_load(CANDLE_STATS_PATH, DEFAULT_PATTERNS)
    score = 0.5; wsum = 0.0
    for p,cnt in patterns.items():
        stat = stats.get(p, {"success_rate":0.5})
        pat_score = stat.get("success_rate",0.5)
        w = min(1.0, cnt)
        score += pat_score * w
        wsum += w
    if wsum > 0: score = score / (1 + wsum)
    return float(max(0.0, min(1.0, score)))

def record_pattern_outcomes(patterns, outcome):
    stats = safe_load(CANDLE_STATS_PATH, DEFAULT_PATTERNS)
    for p,cnt in patterns.items():
        if p not in stats: stats[p] = {"success_rate":0.5,"count":0,"wins":0,"losses":0}
        stats[p]['count'] += cnt
        if outcome: stats[p]['wins'] += cnt
        else: stats[p]['losses'] += cnt
        wins = stats[p]['wins']; losses = stats[p]['losses']; total = wins + losses
        prior=0.5; prior_w=3
        stats[p]['success_rate'] = (wins + prior*prior_w) / (total + prior_w) if total>0 else stats[p]['success_rate']
    safe_save(CANDLE_STATS_PATH, stats)
    return stats

# -------------------------
# Strategy Scoring (0..1)
# -------------------------
def detect_order_blocks(df, lookback=20):
    try:
        recent = df.iloc[-lookback:]
        pct = (recent['close'] - recent['open'])/(recent['open']+1e-9)
        large_bear = (pct < -0.02).any()
        small_range = (recent[-5:]['high'] - recent[-5:]['low']).mean() < (recent['atr'].mean() if 'atr' in recent else 0)*0.6
        return float(bool(large_bear and small_range))
    except:
        return 0.0

def break_of_structure(df):
    try:
        highs = df['high'].rolling(5).max(); lows = df['low'].rolling(5).min()
        if df['close'].iloc[-1] > highs.iloc[-2]: return 1.0
        if df['close'].iloc[-1] < lows.iloc[-2]: return -1.0
    except:
        pass
    return 0.0

def fair_value_gap(df):
    try:
        a = df.iloc[-3]; b = df.iloc[-2]
        if a['high'] < b['low']: return 1.0
        if a['low'] > b['high']: return -1.0
    except:
        pass
    return 0.0

def quantum_engine_v2_score(df):
    try:
        ob = detect_order_blocks(df)
        bos = break_of_structure(df)
        fvg = fair_value_gap(df)
        macd_buy = 1.0 if df['macd'].iloc[-1] > df['macd_signal'].iloc[-1] else 0.0
        ema_trend = 1.0 if df['ema_9'].iloc[-1] > df['ema_21'].iloc[-1] else 0.0
        vol = 1.0 if df['volume'].iloc[-1] > df['volume'].rolling(20).mean().iloc[-1] else 0.0
        raw = 0.20*ob + 0.20*(1.0 if bos>0 else 0.0) + 0.15*(1.0 if fvg>0 else 0.0) + 0.15*macd_buy + 0.15*ema_trend + 0.15*vol
        return float(np.clip(raw,0,1))
    except:
        return 0.5

def momentum_scalper_score(df):
    try:
        vol_mean = df['volume'].rolling(20).mean().iloc[-1] if 'volume' in df else 0.0
        vol_spike = float(df['volume'].iloc[-1] > 1.5*vol_mean) if vol_mean>0 else 0.0
        momentum_break = float((df['close'].iloc[-1] - df['close'].iloc[-5]) / (df['close'].iloc[-5]+1e-9) > 0.008)
        rsi_buy = float(df['rsi'].iloc[-1] < 35) if 'rsi' in df else 0.0
        ema_gc = float(df['ema_9'].iloc[-1] > df['ema_21'].iloc[-1] and df['ema_9'].iloc[-2] <= df['ema_21'].iloc[-2]) if 'ema_9' in df else 0.0
        raw = 0.3*vol_spike + 0.3*momentum_break + 0.2*rsi_buy + 0.2*ema_gc
        return float(np.clip(raw,0,1))
    except:
        return 0.5

def breakout_hunter_score(df):
    try:
        recent_high = df['high'].rolling(20).max().iloc[-2]
        breakout = float(df['close'].iloc[-1] > recent_high)
        vol_conf = float(df['volume'].iloc[-1] > df['volume'].rolling(20).mean().iloc[-1]) if 'volume' in df else 0.0
        bb_break = float(df['close'].iloc[-1] > df['bb_upper'].iloc[-1]) if 'bb_upper' in df else 0.0
        raw = 0.5*breakout + 0.3*vol_conf + 0.2*bb_break
        return float(np.clip(raw,0,1))
    except:
        return 0.5

def mean_reversion_score(df):
    try:
        rsi_low = float(df['rsi'].iloc[-1] < 30) if 'rsi' in df else 0.0
        touch_lower = float(df['close'].iloc[-1] < df['bb_lower'].iloc[-1]) if 'bb_lower' in df else 0.0
        vol_div = float(df['volume'].iloc[-1] < df['volume'].rolling(10).mean().iloc[-1] and ((df['close'].iloc[-1]-df['close'].iloc[-3])<0)) if 'volume' in df else 0.0
        raw = 0.45*rsi_low + 0.35*touch_lower + 0.20*vol_div
        return float(np.clip(raw,0,1))
    except:
        return 0.5

def evaluate_strategies(df):
    return {
        "quantum": quantum_engine_v2_score(df),
        "momentum": momentum_scalper_score(df),
        "breakout": breakout_hunter_score(df),
        "meanreversion": mean_reversion_score(df)
    }

# -------------------------
# Build signals and pick best strategies/timeframes
# -------------------------
def build_signals_for_symbol(client, symbol, tfs=ANALYSIS_TFS):
    signals_by_tf = {}
    latest_price = None; latest_atr = None
    for tf in tfs:
        df = client.get_klines(symbol, tf, limit=200)
        if df is None or df.empty:
            signals_by_tf[tf] = []
            continue
        df = add_institutional_indicators(df)
        strat_scores = evaluate_strategies(df)
        s_list=[]
        for name, score in strat_scores.items():
            sign = 1 if score >= 0.6 else (-1 if score <= 0.4 else 0)
            s_list.append({"name":name, "tf":tf, "signal":sign, "conf": float(score)})
        signals_by_tf[tf]=s_list
        if latest_price is None: latest_price = float(df['close'].iloc[-1])
        if latest_atr is None and 'atr' in df.columns: latest_atr = float(df['atr'].iloc[-1])
    return signals_by_tf, latest_price, latest_atr

def pick_best_strategies(signals_by_tf, top_k=MIN_STRAT_COUNT, threshold=0.8):
    agg=defaultdict(list)
    for tf,slist in signals_by_tf.items():
        for s in slist:
            agg[s['name']].append(s['conf'])
    avg = {k:(sum(v)/len(v) if v else 0.0) for k,v in agg.items()}
    selected = sorted(avg.items(), key=lambda x:x[1], reverse=True)
    picks = [k for k,v in selected if v>=threshold]
    if len(picks) < top_k:
        picks = [k for k,v in selected[:top_k]]
    return picks, avg

def pick_best_timeframes(signals_by_tf, top_k=MIN_TF_COUNT, threshold=0.8):
    tf_scores={}
    for tf,slist in signals_by_tf.items():
        tf_scores[tf] = float(np.mean([s['conf'] for s in slist])) if slist else 0.0
    selected = sorted(tf_scores.items(), key=lambda x:x[1], reverse=True)
    picks = [tf for tf,score in selected if score>=threshold]
    if len(picks) < top_k:
        picks = [tf for tf,score in selected[:top_k]]
    return picks, tf_scores

def analyze_alignment(signals_by_tf, symbol, client):
    tf_comp={}; all_signs=[]
    for tf,slist in signals_by_tf.items():
        if not slist:
            tf_comp[tf]={"sign":0,"conf":0.0}; continue
        ssum = sum(s['signal']*s['conf'] for s in slist)
        tf_sign = 1 if ssum>0.1 else (-1 if ssum < -0.1 else 0)
        tf_conf = float(np.mean([s['conf'] for s in slist]))
        tf_comp[tf]={"sign":tf_sign,"conf":tf_conf}
        if tf_sign!=0: all_signs.append(tf_sign)
    tf_majority_sign=0; tf_agree_frac=0.0
    if all_signs:
        buy=all_signs.count(1); sell=all_signs.count(-1); total=len(all_signs)
        if buy>sell: tf_majority_sign=1; tf_agree_frac=buy/total
        elif sell>buy: tf_majority_sign=-1; tf_agree_frac=sell/total
    # ranges
    RANGE_MAP={"Scalper":["1m","5m"],"Intraday":["15m","30m","1h"],"Swing":["2h","4h","6h","8h","12h"],"Positional":["1d","1w"]}
    range_comp={}
    for r,tfl in RANGE_MAP.items():
        flat=[s for tf in tfl if tf in signals_by_tf for s in signals_by_tf[tf]]
        if not flat:
            range_comp[r]={"sign":0,"conf":0.0}; continue
        rsum=sum(s['signal']*s['conf'] for s in flat)
        rsign = 1 if rsum>0.1 else (-1 if rsum<-0.1 else 0)
        range_comp[r]={"sign":rsign,"conf":float(np.mean([s['conf'] for s in flat]))}
    # candle confidence
    candle_tf = next((tf for tf in ["1h","5m","15m","4h","1d"] if tf in signals_by_tf), None)
    candle_conf=0.5; patterns={}
    if candle_tf:
        df = client.get_klines(symbol, candle_tf, limit=200)
        if df is not None and not df.empty:
            df = add_institutional_indicators(df)
            patterns = detect_patterns_from_df(df)
            candle_conf = candle_confidence_from_patterns(patterns)
    news = {"sentiment": 0.85}  # placeholder: plug API
    return {"tf_comp":tf_comp,"tf_majority_sign":tf_majority_sign,"tf_agree_frac":tf_agree_frac,"range_comp":range_comp,"candle_conf":candle_conf,"candle_patterns":patterns,"news":news}

def compute_tp_sl(entry, sign, atr, rr=2.0, atr_sl_mul=1.0):
    if atr is None or atr<=0 or sign==0: return None, None
    sl_dist = atr*atr_sl_mul
    if sign==1:
        sl = entry - sl_dist; tp = entry + sl_dist*rr
    else:
        sl = entry + sl_dist; tp = entry - sl_dist*rr
    return float(tp), float(sl)

def choose_leverage(selected_tfs):
    top = selected_tfs[0] if selected_tfs else ""
    if top in ["1d","4h","1h","1M","3M","6M","1Y"]:
        return 10
    return 5

def analyze_and_decide(symbol, client=bitget):
    try:
        signals_by_tf, price, atr = build_signals_for_symbol(client, symbol, tfs=ANALYSIS_TFS)
        best_strats, strat_scores = pick_best_strategies(signals_by_tf, top_k=MIN_STRAT_COUNT, threshold=0.8)
        best_tfs, tf_scores = pick_best_timeframes(signals_by_tf, top_k=MIN_TF_COUNT, threshold=0.8)
        alignment = analyze_alignment(signals_by_tf, symbol, client)
        final_sign = alignment['tf_majority_sign']
        meets_tf = alignment['tf_agree_frac'] >= ALIGNMENT_THRESHOLD
        meets_candle = alignment['candle_conf'] >= 0.4
        meets_news = alignment['news']['sentiment'] >= 0.3
        meets_all = (final_sign!=0) and meets_tf and meets_candle and meets_news
        tp, sl = compute_tp_sl(price, final_sign, atr, rr=2.0)
        leverage = choose_leverage(best_tfs)
        confidence = float(np.mean([strat_scores.get(s,0) for s in best_strats])) if best_strats else 0.0
        return {"symbol":symbol,"signals_by_tf":signals_by_tf,"best_strats":best_strats,"best_tfs":best_tfs,"alignment":alignment,"final_sign":final_sign,"meets_all":meets_all,"entry":price,"tp":tp,"sl":sl,"atr":atr,"leverage":leverage,"confidence":confidence,"tf_scores":tf_scores,"strat_scores":strat_scores}
    except Exception as e:
        return {"error":str(e)}

# -------------------------
# Execution wrapper (async)
# -------------------------
async def execute_trade(symbol, qty, decision):
    side = "buy" if decision.get("final_sign")==1 else "sell"
    if not decision.get("meets_all"):
        return {"status":"rejected","reason":"criteria_not_met","details":decision}
    if bitget.demo or not bitget.exchange:
        hist = safe_load(TRADE_HISTORY_PATH, [])
        hist.append({"symbol":symbol,"tf":decision.get("best_tfs"),"strategy":decision.get("best_strats"),"entry":decision.get("entry"),"exit":None,"win":None,"qty":qty,"timestamp":datetime.utcnow().isoformat()})
        safe_save(TRADE_HISTORY_PATH, hist)
        return {"status":"demo","symbol":symbol,"side":side,"qty":qty,"tp":decision.get("tp"),"sl":decision.get("sl"),"leverage":decision.get("leverage")}
    try:
        params = {"leverage": decision.get("leverage",5)}
        res = bitget.create_market_order(symbol, side, qty, params=params)
        hist = safe_load(TRADE_HISTORY_PATH, [])
        hist.append({"symbol":symbol,"tf":decision.get("best_tfs"),"strategy":decision.get("best_strats"),"entry":decision.get("entry"),"exit":None,"win":None,"qty":qty,"order":res,"timestamp":datetime.utcnow().isoformat()})
        safe_save(TRADE_HISTORY_PATH, hist)
        return {"status":"ok","order":res}
    except Exception as e:
        return {"status":"error","error":str(e)}

# -------------------------
# Daily retrain (runs at UTC midnight)
# -------------------------
async def daily_retrain_loop():
    while True:
        now = datetime.now(timezone.utc)
        # compute seconds until next UTC midnight
        next_midnight = (now + timedelta(days=1)).replace(hour=0, minute=0, second=10, microsecond=0)
        wait_sec = (next_midnight - now).total_seconds()
        print(f"[RETRAIN] sleeping {int(wait_sec)}s until next UTC retrain at {next_midnight.isoformat()}")
        await asyncio.sleep(wait_sec)
        try:
            summary = retrain_all()
            print("[RETRAIN] Completed:", summary)
            # optional: send to telegram admin
            if BOT_TOKEN and ADMIN_ID:
                try:
                    await app.bot.send_message(chat_id=ADMIN_ID, text=f"🔄 Daily retrain completed:\n{json.dumps(summary, indent=2)}")
                except:
                    pass
        except Exception as e:
            print("[RETRAIN] error:", e, traceback.format_exc())
        # loop continues to next midnight

# -------------------------
# Telegram Bot handlers & startup
# -------------------------
async def send_authorized(update: Update, text: str):
    uid = update.effective_user.id if update and update.effective_user else None
    if uid in AUTHORIZED_USERS:
        await update.message.reply_text(text)
    else:
        await update.message.reply_text("⛔ You are not authorized to use this command.")

async def analyze_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    symbol = (context.args[0].upper() if context.args else "BTC/USDT")
    await update.message.reply_text(f"🔎 Analyzing {symbol} ...")
    decision = analyze_and_decide(symbol, bitget)
    if "error" in decision:
        await update.message.reply_text(f"❌ Analysis error: {decision['error']}")
        return
    text = f"📊 Analysis {symbol}\nFinal sign: {decision['final_sign']}\nMeets: {decision['meets_all']}\nEntry: {decision['entry']}\nTP: {decision['tp']}\nSL: {decision['sl']}\nLeverage: {decision['leverage']}x\nConfidence: {decision['confidence']:.2f}\nBest TFs: {decision['best_tfs']}\nBest Strats: {decision['best_strats']}"
    await send_authorized(update, text)

async def quantum_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await analyze_command(update, context)  # same for now (quantum is part of evaluation)

async def exec_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    if not context.args or len(context.args) < 2:
        await send_authorized(update, "Usage: /exec SYMBOL QTY")
        return
    symbol = context.args[0].upper()
    qty = float(context.args[1])
    await update.message.reply_text(f"⚡ Preparing to execute {symbol} qty={qty}")
    decision = analyze_and_decide(symbol, bitget)
    res = await execute_trade(symbol, qty, decision)
    await send_authorized(update, f"Execution result: {json.dumps(res, indent=2)}")

async def retrain_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    summary = retrain_all()
    await send_authorized(update, f"🔄 Retrain done:\n{json.dumps(summary, indent=2)}")

async def status_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    demo = bitget.demo
    await send_authorized(update, f"Bot status: running\nBitget demo: {demo}\nSymbols: {len(bitget.list_symbols())}")

async def start_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await send_authorized(update, "🤖 Institutional Ultra AI — ready. Use /analyze SYMBOL or /exec SYMBOL QTY")

# Build app
app = ApplicationBuilder().token(BOT_TOKEN).build()
app.add_handler(CommandHandler("analyze", analyze_command))
app.add_handler(CommandHandler("quantum", quantum_command))
app.add_handler(CommandHandler("exec", exec_command))
app.add_handler(CommandHandler("retrain", retrain_command))
app.add_handler(CommandHandler("status", status_command))
app.add_handler(CommandHandler("start", start_command))

# Start bot polling and background tasks
async def start_bot_and_loops():
    global app
    await app.initialize()
    await app.start()
    print("✅ Telegram bot polling started.")
    # schedule daily retrain
    asyncio.create_task(daily_retrain_loop())
    # main signal loop — scans symbols periodically and sends signals for matches
    asyncio.create_task(main_signal_loop())
    await app.updater.start_polling()
    # keep running
    while True:
        await asyncio.sleep(60)

# Main scanning loop: iterate symbols and produce signals (careful: heavy)
async def main_signal_loop():
    # frequency settings
    SCAN_INTERVAL_SECONDS = 60 * 5   # 5 minutes between scans (tweak)
    SYMBOLS_BATCH_LIMIT = None       # None = all symbols (be careful), or set to an int
    while True:
        try:
            symbols = bitget.list_symbols()
            if SYMBOLS_BATCH_LIMIT:
                symbols = symbols[:SYMBOLS_BATCH_LIMIT]
            # iterate symbols (be mindful of rate limits)
            for sym in symbols:
                try:
                    decision = analyze_and_decide(sym, bitget)
                    # If decision meets criteria, send Telegram signal to authorized users
                    if decision.get("meets_all"):
                        msg = (f"📢 SIGNAL — {sym}\nEntry: {decision['entry']}\nTP: {decision['tp']}\nSL: {decision['sl']}\nLeverage: {decision['leverage']}x\nConfidence: {decision['confidence']:.2f}\nTFs: {decision['best_tfs']}\nStrats: {decision['best_strats']}")
                        for uid in AUTHORIZED_USERS:
                            try:
                                await app.bot.send_message(chat_id=uid, text=msg)
                            except Exception:
                                pass
                        # optionally auto-execute only if OWNER_SECRET matches (security)
                        # (left disabled here to avoid accidental live trades)
                    # small sleep between symbol checks to respect rate limits
                    await asyncio.sleep(0.5)
                except Exception as e:
                    # symbol-specific error, skip
                    # print once
                    print(f"Error analyzing {sym}: {e}")
                    continue
            # wait until next scan
            await asyncio.sleep(SCAN_INTERVAL_SECONDS)
        except Exception as e:
            print("Main scan loop error:", e, traceback.format_exc())
            await asyncio.sleep(10)

# Run app
loop = asyncio.get_event_loop()
loop.create_task(start_bot_and_loops())
print("Institutional Ultra Engine started. Telegram polling + background loops are running.")

⚠️ Bitget running in DEMO mode (no keys).


InvalidToken: You must pass the token you received from https://t.me/Botfather!